## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

In [ ]:
# View the column names in the data
df.columns


In [ ]:
# Use the describe function to gather some basic statistics
df.describe()

In [ ]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df["category"].isnull().count()



In [ ]:
# What three item categories had the most entries?
top_3_categories  = df["category"].value_counts().nlargest(3)
top_3_categories_2 = df.groupby('category').size().nlargest(3)
print(top_3_categories )
print(top_3_categories_2 )

In [ ]:
# For the category with the most entries,
# which subcategory had the most entries?df
df_consumables =  df[df["category"] == 'consumables']
top_Subcateg0ry_consumables = df_consumables.groupby('subcategory').size().nlargest(1)
print(top_Subcateg0ry_consumables)


In [ ]:
# Which five clients had the most entries in the data?
top5_Clients = df.groupby('client_id').size().nlargest(5)
top5_Clients


In [ ]:
# Store the client ids of those top 5 clients in a list.
top5_Clients_List = top5_Clients.index.tolist()
top5_Clients_List

In [ ]:
# How many total units (the qty column) did the
# client with the most entries order order?
client_entries = df['client_id'].value_counts()
client_most_entries = client_entries.idxmax()
total_units_most_entries = df[df['client_id'] == client_most_entries]['qty'].sum()
total_units_most_entries

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [ ]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["line_subtotal"] = df["unit_price"] * df["qty"]
df[['unit_price', 'qty', 'line_subtotal']].head()

In [ ]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.

# Define a lambda function with if condition to calculate the field based on existing columns

df["total_weight"] = df["unit_weight"]  * df["qty"] 

calculate_Shipping_price = lambda row: row['total_weight'] * 7  if row['total_weight'] >= 50 else row['total_weight'] * 10 

df["shipping_price"] = df.apply(calculate_Shipping_price , axis=1)

df[['unit_price','unit_weight','qty','total_weight','shipping_price']].head()


In [ ]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

calculate_Total_price = lambda row: round((row['line_subtotal'] + row['shipping_price']) * (1.0925),2)
df["line_price"] = df.apply(calculate_Total_price, axis=1)
df[['line_subtotal','shipping_price','line_price']].head(5)


In [ ]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
df["line_cost"] = df["unit_cost"] * df["qty"] +  df["shipping_price"]
df[['shipping_price','line_price','unit_cost','qty','line_cost']].head(5)


In [ ]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit']=df['line_price']-df['line_cost']

df[['client_id','qty','line_price','line_cost','line_profit']].head(5)


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [ ]:
# Che()ck your work using the totals above
print(f"Order ID 2742071 had a total price of ${df.query('order_id == 2742071')['line_profit'].sum().round(2) } " )

print(f"Order ID 2173913 had a total price of ${df.query('order_id == 2173913')['line_profit'].sum().round(2) } " )

print(f"Order ID 6128929 had a total price of ${df.query('order_id == 6128929')['line_profit'].sum().round(2) } " )

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [72]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
top5_Client = df.loc[df['client_id'].isin(top5_Clients_List)]
top5_CLient_Byquanity =  top5_Client.groupby('client_id').agg({'line_price':'sum'}).sort_values(by="client_id", ascending=False).style.format("${:,.2f}")
top5_CLient_Byquanity


,line_price
client_id,
66037,"$10,259,514.80"
46820,"$9,743,794.33"
38378,"$12,906,550.87"
33615,"$8,377,144.66"
24741,"$82,268,892.04"


In [115]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

top5_Client_Summery =  top5_Client.groupby('client_id').agg({
                                    'qty': 'sum',
                                    'shipping_price': 'sum',
                                    'line_price': 'sum',
                                    'line_cost': 'sum',
                                    'line_profit' :'sum'
                                }).sort_values('line_profit',ascending=False)

top5_Client_Summery


,qty,shipping_price,line_price,line_cost,line_profit
client_id,,,,,
24741,239862,5126448.37,82268892.04,45688899.71,36579992.33
38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
66037,43018,1395151.85,10259514.80,7004482.98,3255031.82
46820,75768,1601448.84,9743794.33,7007191.64,2736602.69
33615,64313,1828834.89,8377144.66,6175163.91,2201980.75


In [116]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

column_To_Format = ['line_price','line_cost','line_profit','shipping_price']

def currency_format(amount):
    return f"${amount/1000000:.2f}M"

for col in column_To_Format:
    top5_Client_Summery[col] = top5_Client_Summery[col].apply(currency_format)

top5_Client_Summery.columns=['Units','Shipping','Total Revenue','Total Cost','Total Profit']
top5_Client_Summery


,Units,Shipping,Total Revenue,Total Cost,Total Profit
client_id,,,,,
24741,239862,$5.13M,$82.27M,$45.69M,$36.58M
38378,73667,$3.43M,$12.91M,$9.63M,$3.27M
66037,43018,$1.40M,$10.26M,$7.00M,$3.26M
46820,75768,$1.60M,$9.74M,$7.01M,$2.74M
33615,64313,$1.83M,$8.38M,$6.18M,$2.20M


In [117]:
# Sort the updated data by "Total Profit" form highest to lowest
top5_Client_Summery['ntp']= pd.to_numeric(top5_Client_Summery['Total Profit'].str.replace('$','').str.replace('M',''))

top5_Client_Summery.sort_values('ntp',ascending=False).filter(['Client ID','Units','Shipping','Total Revenue','Total Cost','Total Profit'])

,Units,Shipping,Total Revenue,Total Cost,Total Profit
client_id,,,,,
24741,239862,$5.13M,$82.27M,$45.69M,$36.58M
38378,73667,$3.43M,$12.91M,$9.63M,$3.27M
66037,43018,$1.40M,$10.26M,$7.00M,$3.26M
46820,75768,$1.60M,$9.74M,$7.01M,$2.74M
33615,64313,$1.83M,$8.38M,$6.18M,$2.20M
